In [ ]:
import io
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path
import pickle
import PIL
import torch
import torchvision
import urllib
import warnings
import sys

sys.path.append(os.path.join(os.path.dirname(os.path.realpath('__file__')), '..'))
from attacks import eot_attacks
from attacks import utils

from utils import labels_util
from utils import datasets

# Disable annoying UserWarning caused by using nn.Upsample
# in the relighting model.
warnings.filterwarnings("ignore", category=UserWarning) 

%load_ext autoreload
%autoreload 2

In [ ]:
config = {}
config['classif_model_name'] = 'pubfig_facenet'
config['relight_model_name'] = 'multi_illumination_murmann'
config['relighter_eps'] = 1e-4

# checkpoint_path = '/nfs/homedirs/musat/adversarial-attacks-by-relighting/new_repo/project-1/'
checkpoint_path = '/export/home/acs/stud/a/andreea.musat/prakt/adversarial-attacks-by-relighting/latest_repo/project-1/'
checkpoint_path += 'relighters/multi_illumination/checkpoints/relight/epoch_13.pth'

relight_model = utils.load_relighting_model(config['relight_model_name'], checkpoint_path)
classif_model = utils.load_classification_model(config['classif_model_name'])

idx_to_label = labels_util.load_idx_to_label('pubfig10')
label_to_idx = {label : idx for idx, label in idx_to_label.items()}

In [ ]:
data_path = '../data/pubfig/PubFig_test'

config['attack_type'] = 'class_constrained_eot'
config['target_label'] = 0 
config['debugging'] = False
config['num_iterations'] = 5
config['learning_rate'] = 0.1
config['gamma'] = 1.3
config['eps'] = 0.02

In [ ]:
dataset = datasets.PubFigDataset('../data/pubfig/', mode='test', crop=False)

for idx in range(1, len(dataset)):
    img, gt_label = dataset[idx]
    
    config['gt_label'] = gt_label
    if gt_label == config['target_label']:
        config['target_label'] = 8

    result = eot_attacks.do_attack(relight_model, classif_model, img, config)
    
    print(gt_label, result['orig_label'])
    
    if 'root_img' in result:
        utils.visualize_attack(img, result, idx_to_label)